FETCH DATA

In [51]:
# imports

import requests
import pandas as pd

In [52]:
# define api key and year of data

API_KEY = "82c05d5ce225ec3ac96cbf6cf4e47e92aedecd1c"
year = 2022

In [53]:
# define which variables to grab from dataset

variables = [
    # Demographics
    "B01001_001E", "B01001_002E", "B01001_026E", "B01002_001E",
    # Education
    "B15003_001E", "B15003_022E", "B15003_023E", "B15003_024E", "B15003_025E",
    # Income
    "B19013_001E", "B19301_001E", "B17001_002E",
    # Employment
    "B23025_003E", "B23025_005E", 
    # Housing
    "B25077_001E", "B25002_002E", "B25014_001E", "B25003_002E", "B25003_003E",
    # Health proxy
    "B18101_001E", "B18101_002E", "B18101_003E", "B18101_004E"
]

In [ ]:
# grab data from dataset and place into pandas dataframe

vars_string = ",".join(variables)

url = f"https://api.census.gov/data/{year}/acs/acs5?get={vars_string}&for=state:*&key={API_KEY}"

response = requests.get(url)

if response.status_code == 200:
    data=response.json()
    columns=data[0]
    values=data[1:]
    df=pd.DataFrame(values, columns=columns)
    print(df.head())
else:
    print(f"Error: {response.status_code}-{response.text}")

  B01001_001E B01001_002E B01001_026E B01002_001E B15003_001E B15003_022E  \
0     5028092     2445995     2582097        39.3     3428520      572252   
1      734821      386649      348172        35.3      485871       93744   
2     7172282     3588510     3583772        38.4     4878959      958447   
3     3018669     1491622     1527047        38.4     2031847      317437   
4    39356104    19708947    19647157        37.3    26842698     5935292   

  B15003_023E B15003_024E B15003_025E B19013_001E  ... B25077_001E  \
0      261039       56574       42980       59609  ...      179400   
1       39645        9589        6419       86370  ...      318000   
2      429317       93897       69793       72581  ...      321400   
3      134754       29822       19966       56335  ...      162400   
4     2515475      687281      488925       91905  ...      659300   

  B25002_002E B25014_001E B25003_002E B25003_003E B18101_001E B18101_002E  \
0     1933150     1933150     1347792  

DATA CLEANING & PREPROCESSING

In [55]:
# rename columns

# define labels
label_map = {
    # Demographics
    "B01001_001E": "Total Population",
    "B01001_002E": "Male Population",
    "B01001_026E": "Female Population",
    "B01002_001E": "Median Age",

    # Education (Population 25+)
    "B15003_001E": "Population 25+ (Total)",
    "B15003_022E": "Bachelor's Degree",
    "B15003_023E": "Master's Degree",
    "B15003_024E": "Professional Degree",
    "B15003_025E": "Doctorate Degree",

    # Income
    "B19013_001E": "Median Household Income",
    "B19301_001E": "Per Capita Income",
    "B17001_002E": "Individuals Below Poverty",

    # Employment
    "B23025_003E": "Civilian Labor Force",
    "B23025_005E": "Unemployed Persons",

    # Housing
    "B25077_001E": "Median Home Value",
    "B25002_002E": "Total Housing Units",
    "B25014_001E": "Occupied Housing Units (Total)",
    "B25003_002E": "Owner-Occupied Housing Units",
    "B25003_003E": "Renter-Occupied Housing Units",

    # Health proxy (Disability data)
    "B18101_001E": "Total Noninstitutionalized Population (16+)",
    "B18101_002E": "Population With Any Disability",
    "B18101_003E": "Population With Hearing Difficulty",
    "B18101_004E": "Population With Vision Difficulty"
}

# rename cols
df = df.rename(columns=label_map)

In [56]:
# check
print(df.head())

  Total Population Male Population Female Population Median Age  \
0          5028092         2445995           2582097       39.3   
1           734821          386649            348172       35.3   
2          7172282         3588510           3583772       38.4   
3          3018669         1491622           1527047       38.4   
4         39356104        19708947          19647157       37.3   

  Population 25+ (Total) Bachelor's Degree Master's Degree  \
0                3428520            572252          261039   
1                 485871             93744           39645   
2                4878959            958447          429317   
3                2031847            317437          134754   
4               26842698           5935292         2515475   

  Professional Degree Doctorate Degree Median Household Income  ...  \
0               56574            42980                   59609  ...   
1                9589             6419                   86370  ...   
2          

In [57]:
# check for empty values in states 
print(df['state'].isna().sum())

# check for unique values
print(sorted(df['state'].unique()))

#check for any extra white space or weird entries, blank
print(df[df['state'].str.strip()==''])


0
['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '72']
Empty DataFrame
Columns: [Total Population, Male Population, Female Population, Median Age, Population 25+ (Total), Bachelor's Degree, Master's Degree, Professional Degree, Doctorate Degree, Median Household Income, Per Capita Income, Individuals Below Poverty, Civilian Labor Force, Unemployed Persons, Median Home Value, Total Housing Units, Occupied Housing Units (Total), Owner-Occupied Housing Units, Renter-Occupied Housing Units, Total Noninstitutionalized Population (16+), Population With Any Disability, Population With Hearing Difficulty, Population With Vision Difficulty, state]
Index: []

[0 rows x 24 columns]


In [58]:
# change state ints to state abbreviations
df['state'] = df['state'].astype(str).str.zfill(2) # make into string and pad with leading 0s

fips_to_abbrev = {
    '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA',
    '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL',
    '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN',
    '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME',
    '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS',
    '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH',
    '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND',
    '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI',
    '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT',
    '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI',
    '56': 'WY', '72': 'PR'
}

df['state_abbrev'] = df['state'].map(fips_to_abbrev)

In [59]:
# check 
print(df.head())

  Total Population Male Population Female Population Median Age  \
0          5028092         2445995           2582097       39.3   
1           734821          386649            348172       35.3   
2          7172282         3588510           3583772       38.4   
3          3018669         1491622           1527047       38.4   
4         39356104        19708947          19647157       37.3   

  Population 25+ (Total) Bachelor's Degree Master's Degree  \
0                3428520            572252          261039   
1                 485871             93744           39645   
2                4878959            958447          429317   
3                2031847            317437          134754   
4               26842698           5935292         2515475   

  Professional Degree Doctorate Degree Median Household Income  ...  \
0               56574            42980                   59609  ...   
1                9589             6419                   86370  ...   
2          

In [60]:
# check which columns have null values accross the dataset (perfect for integers and enough for it)

print(df.isnull().sum())

Total Population                               0
Male Population                                0
Female Population                              0
Median Age                                     0
Population 25+ (Total)                         0
Bachelor's Degree                              0
Master's Degree                                0
Professional Degree                            0
Doctorate Degree                               0
Median Household Income                        0
Per Capita Income                              0
Individuals Below Poverty                      0
Civilian Labor Force                           0
Unemployed Persons                             0
Median Home Value                              0
Total Housing Units                            0
Occupied Housing Units (Total)                 0
Owner-Occupied Housing Units                   0
Renter-Occupied Housing Units                  0
Total Noninstitutionalized Population (16+)    0
Population With Any 

In [61]:
# check for empty strings or whitespace-only strings (only good for string values)

for col in df.select_dtypes(include='object').columns:
    empty_count = df[col].str.strip().eq('').sum()
    if empty_count > 0:
        print(f"Column '{col}' has {empty_count} empty/blank strings(s)") 

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 25 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   Total Population                             52 non-null     object
 1   Male Population                              52 non-null     object
 2   Female Population                            52 non-null     object
 3   Median Age                                   52 non-null     object
 4   Population 25+ (Total)                       52 non-null     object
 5   Bachelor's Degree                            52 non-null     object
 6   Master's Degree                              52 non-null     object
 7   Professional Degree                          52 non-null     object
 8   Doctorate Degree                             52 non-null     object
 9   Median Household Income                      52 non-null     object
 10  Per Capita Incom

In [63]:
print(df.describe())

       Total Population Male Population Female Population Median Age  \
count                52              52                52         52   
unique               52              52                52         35   
top             5028092         2445995           2582097       39.9   
freq                  1               1                 1          4   

       Population 25+ (Total) Bachelor's Degree Master's Degree  \
count                      52                52              52   
unique                     52                52              52   
top                   3428520            572252          261039   
freq                        1                 1               1   

       Professional Degree Doctorate Degree Median Household Income  ...  \
count                   52               52                      52  ...   
unique                  52               52                      52  ...   
top                  56574            42980                   59609  ...   

In [64]:
list(df.columns)

['Total Population',
 'Male Population',
 'Female Population',
 'Median Age',
 'Population 25+ (Total)',
 "Bachelor's Degree",
 "Master's Degree",
 'Professional Degree',
 'Doctorate Degree',
 'Median Household Income',
 'Per Capita Income',
 'Individuals Below Poverty',
 'Civilian Labor Force',
 'Unemployed Persons',
 'Median Home Value',
 'Total Housing Units',
 'Occupied Housing Units (Total)',
 'Owner-Occupied Housing Units',
 'Renter-Occupied Housing Units',
 'Total Noninstitutionalized Population (16+)',
 'Population With Any Disability',
 'Population With Hearing Difficulty',
 'Population With Vision Difficulty',
 'state',
 'state_abbrev']

In [65]:
df_numeric = df.apply(pd.to_numeric, errors='coerce')  # convert all to numbers, non-convertible become NaN
min_vals = df_numeric.min()
print(min_vals)

Total Population                               577929.0
Male Population                                295835.0
Female Population                              282094.0
Median Age                                         31.4
Population 25+ (Total)                         391802.0
Bachelor's Degree                               71313.0
Master's Degree                                 31053.0
Professional Degree                              6077.0
Doctorate Degree                                 5138.0
Median Household Income                         24002.0
Per Capita Income                               15637.0
Individuals Below Poverty                       60134.0
Civilian Labor Force                           299351.0
Unemployed Persons                              11456.0
Median Home Value                              121800.0
Total Housing Units                            234156.0
Occupied Housing Units (Total)                 234156.0
Owner-Occupied Housing Units                   1

In [ ]:
# Feature Engineering

cols_to_convert = df.columns.difference(['state_abbrev'])
df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# Demographics
df['Male (%) of Population'] = (df['Male Population']/df['Total Population']) * 100
df['Female (%) of Population'] = (df['Female Population']/df['Total Population']) * 100
df['(%) of Population 25+'] = (df['Population 25+ (Total)']/df['Total Population']) * 100

# Education
df['(%) with Bachelor\'s Degree'] = (df["Bachelor's Degree"]/df['Total Population'])*100 
df['(%) with Master\'s Degree'] = (df["Master's Degree"]/df['Total Population'])*100
df['(%) with Doctorate Degree'] = (df["Doctorate Degree"]/df['Total Population'])*100
df['(%) with Professional Degree'] = (df["Professional Degree"]/df['Total Population'])*100

# Workforce
df['(%) of Unemployed Persons'] = (df['Unemployed Persons']/df['Total Population'])* 100
df['Employed Persons'] = df['Total Population'] - df['Unemployed Persons']
df['Employment Rate'] = (df['Employed Persons']/df['Civilian Labor Force']) * 100

# Home Ownershio
df['Home Ownership Rate (%)'] = (df['Owner-Occupied Housing Units']/df['Total Housing Units']) * 100
df['Renting Rate (%)'] = (df['Renter-Occupied Housing Units']/df['Total Housing Units']) * 100

# Adjusting (%)s
df['Home Ownership Rate (%)'] = df['Home Ownership Rate (%)'].round(2)
df['Renting Rate (%)'] = df['Renting Rate (%)'].round(2)
df['Employment Rate'] = df['Employment Rate'].round(2)
df['(%) with Bachelor\'s Degree'] = df['(%) with Bachelor\'s Degree'].round(2)
df['(%) with Master\'s Degree'] = df['(%) with Master\'s Degree'].round(2)
df['(%) with Doctorate Degree'] = df['(%) with Doctorate Degree'].round(2)
df['(%) with Professional Degree'] = df['(%) with Professional Degree'].round(2)
df['Male (%) of Population'] = df['Male (%) of Population'].round(2)
df['Female (%) of Population'] = df['Female (%) of Population'].round(2)
df['(%) of Population 25+'] = df['(%) of Population 25+'].round(2)
df['(%) of Unemployed Persons'] = df['(%) of Unemployed Persons'].round(2)


In [67]:
df.head()

,Total Population,Male Population,Female Population,Median Age,Population 25+ (Total),Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree,Median Household Income,...,(%) of Population 25+,(%) with Bachelor's Degree,(%) with Master's Degree,(%) with Doctorate Degree,(%) with Professional Degree,(%) of Unemployed Persons,Employed Persons,Employment Rate,Home Ownership Rate (%),Renting Rate (%)
0,5028092,2445995,2582097,39.3,3428520,572252,261039,56574,42980,59609,...,68.19,11.38,5.19,0.85,1.13,2.39,4908062,210.67,69.72,30.28
1,734821,386649,348172,35.3,485871,93744,39645,9589,6419,86370,...,66.12,12.76,5.40,0.87,1.30,3.13,711786,196.52,66.27,33.73
2,7172282,3588510,3583772,38.4,4878959,958447,429317,93897,69793,72581,...,68.03,13.36,5.99,0.97,1.31,2.59,6986224,201.49,66.27,33.73
3,3018669,1491622,1527047,38.4,2031847,317437,134754,29822,19966,56335,...,67.31,10.52,4.46,0.66,0.99,2.37,2947068,211.85,66.23,33.77
4,39356104,19708947,19647157,37.3,26842698,5935292,2515475,687281,488925,91905,...,68.20,15.08,6.39,1.24,1.75,3.26,38074049,190.26,55.63,44.37


In [ ]:
# Save the processed data